In [92]:
from keras import layers
import tensorflow as tf
import pandas as pd
import numpy as np

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)


df = pd.read_csv('../csv_dateien/starcraftFinalcsvs/allRunsButOnlyTerran.csv')

conditions = [
    (df['fraction'] == "Protoss"), #1
    (df['fraction'] == "Terraner"), #2
    (df['fraction'] == "Zerg"), #3
    (df['fraction'] == "T"), #2
]

values = [1, 2, 3, 2]

df["fraction"] = np.select(conditions, values)
df=df[df["total_army"]>0]
df=df[df["fraction"]==2]
df=df[df["supply"]<=200]

dfPrepSample = df.drop(columns=["player","winner","replay_filename", "Unnamed: 0", "total_army_value","fraction"])
len(dfPrepSample)


3009

In [93]:
df_sample = dfPrepSample.sample(frac=1, random_state=12)
dfTrainSample = df_sample.sample(frac=0.8)

dfTrainSample.head()

,total_workers,resource_mining,total_army,supply,time
4276,72,16930,50.0,141.0,678
3457,34,22475,68.0,134.0,744
3646,67,20336,67.0,157.0,759
5492,72,21758,33.0,117.0,639
5022,30,7057,13.0,46.0,345


In [94]:
dfTestSample = df_sample.drop(dfTrainSample.index)

dfTestSample.head()

,total_workers,resource_mining,total_army,supply,time
3989,14,1815,1.0,23.0,174
1438,55,44034,79.0,149.0,989
5806,47,10825,32.0,94.0,538
5137,62,17451,53.0,133.0,658
4010,18,1373,1.0,23.0,174


In [95]:
dfTrainFeatures = dfTrainSample.copy()
dfTrainFeatures

,total_workers,resource_mining,total_army,supply,time
4276,72,16930,50.0,141.0,678
3457,34,22475,68.0,134.0,744
3646,67,20336,67.0,157.0,759
5492,72,21758,33.0,117.0,639
5022,30,7057,13.0,46.0,345
...,...,...,...,...,...
3604,26,11479,17.0,47.0,437
19,35,9595,26.0,62.0,449
5156,76,42562,73.0,164.0,1006
3482,58,34015,69.0,157.0,834


In [96]:

dfTrainLabels = dfTrainFeatures.pop("supply")
dfTrainFeatures = np.array(dfTrainFeatures)
dfTrainFeatures

array([[   72., 16930.,    50.,   678.],
       [   34., 22475.,    68.,   744.],
       [   67., 20336.,    67.,   759.],
       ...,
       [   76., 42562.,    73.,  1006.],
       [   58., 34015.,    69.,   834.],
       [   17.,  1569.,     1.,   176.]])

In [97]:
# normalize = layers.Normalization()
# normalize.adapt(dfTrainFeatures)

In [98]:
sc_model = tf.keras.Sequential([layers.Dense(5, activation="relu"),layers.Dense(3, activation="relu"),layers.Dense(3, activation="relu"), layers.Dense(1)]) # letzte lösung 30/5

sc_model.compile(loss = tf.keras.losses.MeanSquaredError(),
                      optimizer = tf.keras.optimizers.Adam())

In [99]:
sc_model.fit(dfTrainFeatures, dfTrainLabels, epochs=1000)  #350

Epoch 1/1000
76/76 [==============================] - 1s 733us/step - loss: 13424.5762
Epoch 2/1000
76/76 [==============================] - 0s 699us/step - loss: 13408.5068
Epoch 3/1000
76/76 [==============================] - 0s 653us/step - loss: 13392.4590
Epoch 4/1000
76/76 [==============================] - 0s 687us/step - loss: 13376.4443
Epoch 5/1000
76/76 [==============================] - 0s 680us/step - loss: 13360.4219
Epoch 6/1000
76/76 [==============================] - 0s 714us/step - loss: 13344.4336
Epoch 7/1000
76/76 [==============================] - 0s 700us/step - loss: 13328.4678
Epoch 8/1000
76/76 [==============================] - 0s 720us/step - loss: 13312.5225
Epoch 9/1000
76/76 [==============================] - 0s 673us/step - loss: 13296.5635
Epoch 10/1000
76/76 [==============================] - 0s 667us/step - loss: 13280.6270
Epoch 11/1000
76/76 [==============================] - 0s 687us/step - loss: 13264.7842
Epoch 12/1000
76/76 [====================

In [100]:
sc_model.summary()


Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_32 (Dense)            (None, 5)                 25        
                                                                 
 dense_33 (Dense)            (None, 3)                 18        
                                                                 
 dense_34 (Dense)            (None, 3)                 12        
                                                                 
 dense_35 (Dense)            (None, 1)                 4         
                                                                 
Total params: 59
Trainable params: 59
Non-trainable params: 0
_________________________________________________________________


In [101]:
dfTestFeatures = dfTestSample.copy()

dfTestLabels = dfTestFeatures.pop("supply")
dfTestFeatures = np.array(dfTestFeatures)
dfTestLabels.head()

3989     23.0
1438    149.0
5806     94.0
5137    133.0
4010     23.0
Name: supply, dtype: float64

In [102]:
from sklearn.metrics import r2_score


predictedValues = sc_model.predict(dfTestFeatures)
r2_score_sc = (r2_score(dfTestLabels, predictedValues))
stri = ""
if(r2_score_sc<0):
    stri = "negative" + str(r2_score_sc)
else:
    stri = str(r2_score_sc)

print(f"The R^2 value in the test set is {stri}")

19/19 [==============================] - 0s 676us/step
The R^2 value in the test set is negative-0.4454885768014749


In [103]:
# sc_model.save('sc_ts_model.h5')
